### Back Translation : A data augmentation technique to over-sample maladaptive posts.

In [ ]:
!pip install transformers
!pip install mosestokenizer
!pip install sentencepiece
from transformers import MarianMTModel, MarianTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Get the name of the first model
first_model_name = 'Helsinki-NLP/opus-mt-en-fr'

# Get the tokenizer
first_model_tkn = MarianTokenizer.from_pretrained(first_model_name)

# Load the pretrained model based on the name
first_model = MarianMTModel.from_pretrained(first_model_name)

/usr/local/lib/python3.9/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
# Get the name of the second model
second_model_name = 'Helsinki-NLP/opus-mt-fr-en'

# Get the tokenizer
second_model_tkn = MarianTokenizer.from_pretrained(second_model_name)

# Load the pretrained model based on the name
second_model = MarianMTModel.from_pretrained(second_model_name)

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("MalAdapTrain.csv", encoding="ISO-8859-1")

In [ ]:
df.head()

In [ ]:
df1 = df[df['Adaptive'] == 0]

In [ ]:
df2 = df[df['Adaptive'] == 1]

In [ ]:
df2.shape

(946, 3)

In [ ]:
df1.shape

(243, 3)

In [ ]:
original_texts = df1['Text'][201:243].tolist()
original_texts

['A man is coughing immensely in a packed train.. Others are looking worried about it, and one of the passengers asks: \n "Excuse me, do you have coronavirus?" \n "No sir, I\'m diagnosed with overt tuberculosis." \n "Thank God", the others sigh in relief.',
 'I told a joke about Coronavirus and nobody laughed. Except that Chinese kid in the back. He got it.',
 "Trump 'could suck coronavirus out of 60,000 people' and he'd still be criticized â\x80\x93 Huckabee",
 "Yo mama's so ugly. She took off her facemask during quarantine and was arrested for indecent exposure",
 'This quarantine thing is not working out.. My wife and I are becoming friends and I almost told her about my girlfriend.',
 "my girlfriend has been quarantined because of the coronavirus.. i'm really worried, she's stuck at her ex's house and sent a voicemail of her moaning in pain. hopefully god is good to her and she can come home soon.",
 'You may not have a low iq. you could just be a narcistic, self centered, asshole 

In [ ]:
def format_batch_texts(language_code, batch_texts):
  
    formated_bach = [">>{}<< {}".format(language_code, text) for text in batch_texts]

    return formated_bach

In [ ]:
# Test of the function
format_batch_texts("fr", original_texts)

['>>fr<< A man is coughing immensely in a packed train.. Others are looking worried about it, and one of the passengers asks: \n "Excuse me, do you have coronavirus?" \n "No sir, I\'m diagnosed with overt tuberculosis." \n "Thank God", the others sigh in relief.',
 '>>fr<< I told a joke about Coronavirus and nobody laughed. Except that Chinese kid in the back. He got it.',
 ">>fr<< Trump 'could suck coronavirus out of 60,000 people' and he'd still be criticized â\x80\x93 Huckabee",
 ">>fr<< Yo mama's so ugly. She took off her facemask during quarantine and was arrested for indecent exposure",
 '>>fr<< This quarantine thing is not working out.. My wife and I are becoming friends and I almost told her about my girlfriend.',
 ">>fr<< my girlfriend has been quarantined because of the coronavirus.. i'm really worried, she's stuck at her ex's house and sent a voicemail of her moaning in pain. hopefully god is good to her and she can come home soon.",
 '>>fr<< You may not have a low iq. you c

In [ ]:
def perform_translation(batch_texts, model, tokenizer, language="fr"):
    # Prepare the text data into appropriate format for the model
    formated_batch_texts = format_batch_texts(language, batch_texts)
    
    # Generate translation using model
    translated = model.generate(**tokenizer(formated_batch_texts, return_tensors="pt", padding = True))

    # Convert the generated tokens indices back into text
    translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    
    return translated_texts

In [ ]:
translated_texts = perform_translation(original_texts, first_model, first_model_tkn)

In [ ]:
translated_texts

['Un homme tousse immensément dans un train chargé.. D\'autres sont inquiets à ce sujet, et l\'un des passagers demande: "Excusez-moi, avez-vous le coronavirus?" "Non monsieur, je suis diagnostiqué avec la tuberculose ouverte." "Merci Dieu", les autres soupirent en soulagement.',
 "J'ai raconté une blague sur le Coronavirus et personne n'a ri, sauf ce Chinois dans le dos.",
 'Trump \'pourrait sucer le coronavirus sur 60 000 personnes\' et il \'était encore critiqué \'" Huckabee',
 'Yo maman est si laid. Elle a enlevé son masque en quarantaine et a été arrêtée pour exposition indécente.',
 'Cette chose de quarantaine ne fonctionne pas.. Ma femme et moi devenons amis et je lui ai presque parlé de ma petite amie.',
 "Ma copine a été mise en quarantaine à cause du coronavirus.. Je suis vraiment inquiet, elle est coincée chez son ex et a envoyé un message vocal de ses gémissements dans la douleur. J'espère que Dieu est bon pour elle et elle peut rentrer bientôt à la maison.",
 "Vous n'avez 

In [ ]:
back_translated_texts = perform_translation(translated_texts, second_model, second_model_tkn)

In [ ]:
back_translated_texts

['A man coughs immensely on a loaded train. Others are worried about it, and one of the passengers asks: "Excuse me, do you have the coronavirus?" "No sir, I am diagnosed with open tuberculosis." "Thank God," the others sighed in relief.',
 'I told a joke about Coronavirus and nobody laughed, except that Chinese guy in the back.',
 'Trump \'could suck the coronavirus on 60,000 people\' and he \'was still criticized\' "Huckabee',
 "Yo mommy's so ugly. She took off her quarantine mask and was arrested for indecent exposure.",
 'My wife and I are becoming friends and I almost told her about my girlfriend.',
 "My girlfriend was quarantined because of the coronavirus.. I'm really worried, she's stuck at her ex and sent a voice message of her groaning in pain. I hope God is good for her and she can come home soon.",
 "Maybe you don't have a iq down. You could just be a narcist, self-centered, asshole who thinks only of themselves. Who knows or cares.",
 '"The worst hasn\'t happened yet. - Wh